# Importing Libraries

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd

# Getting The Dataset

In [ ]:
df = pd.read_csv("../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv")

# Getting Analysis of Dataframe

In [ ]:
df.head()

In [ ]:
df['Class'].value_counts()

In [ ]:
df['Class'].value_counts(sort=True).plot.bar()

# Getting the Images Folder Path

In [ ]:
train_image = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train'
test_image = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test'

# Setting Input Image Size

In [ ]:
image_size = [299, 299]

# Splitting Dataframe

In [ ]:
train_df,valid_df = train_test_split(df,test_size=.15,stratify=df.Class.values,shuffle=True)


train_df.reset_index(inplace=True,drop=True)
valid_df.reset_index(inplace=True,drop=True)

## Training Dataframe

In [ ]:
train_df.head()

In [ ]:
train_df['Class'].value_counts()

In [ ]:
train_df['Class'].value_counts(sort=True).plot.bar()

## Validation Dataframe

In [ ]:
valid_df.head()

In [ ]:
valid_df['Class'].value_counts()

In [ ]:
valid_df['Class'].value_counts(sort=True).plot.bar()

# Getting Images and Preprocesing 

## Getting Images for training

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_dataframe(dataframe=train_df, directory=train_image,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(299,299), batch_size=32)

# Getting Images for Vaidation

In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

validation_set = validation_datagen.flow_from_dataframe(dataframe=valid_df, directory=train_image,
                                                 x_col="Image", y_col="Class",
                                                 class_mode="categorical",
                                                 target_size=(299,299), batch_size=32)

# Getting The InceptionResNetV2 Model

In [ ]:
incep = InceptionResNetV2(input_shape=image_size + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in incep.layers:
  layer.trainable = False  

x = Flatten()(incep.output)


prediction = Dense(6, activation='softmax')(x)

# create a model object
model = Model(inputs=incep.input, outputs=prediction)

# view the structure of the model
model.summary()

# Compile the model

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

# Fitting the model

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=validation_set,
  epochs=12,
  steps_per_epoch=training_set.n//32,
  validation_steps=validation_set.n//32
)

# Ploting Loss ansd Accuracy

In [ ]:
# Loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


# Getting Predictions for Test Data Images

In [ ]:
name=[]
y_pred=[]
labels = (training_set.class_indices)
labels = dict((v,k) for k,v in labels.items())
for i in os.listdir('../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'):
    name.append(i)
    i='../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'+i
    img=image.load_img(i,target_size=(299,299,3))
    img=image.img_to_array(img)/255
    pred=model.predict(img.reshape(1,299,299,3))
    y_pred.append(labels[np.argmax(pred[0])])
    
data=pd.DataFrame((zip(name,y_pred)),columns=['Image','Class'])
data.head()

In [ ]:
data.to_csv('my_submission.csv',index=False)

In [ ]:
data.shape